In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ball_by_ball = pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv')
matches = pd.read_csv('../input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv')

ball_by_ball=ball_by_ball.sort_values(by=['id','inning','over','ball'])

#Get the opening batsman for each match
opening_batsmen=ball_by_ball.query("over==0 and ball==1").drop_duplicates(['id','inning','batsman','non_striker'])[['id','inning','batsman','non_striker']]
new_ball_by_ball = pd.merge(ball_by_ball, opening_batsmen,  how='inner', left_on=['id','inning'], right_on = ['id','inning'])
new_ball_by_ball['opening_bat']=new_ball_by_ball['batsman_y']+new_ball_by_ball['non_striker_y']
new_ball_by_ball['opening_bat_rev']=new_ball_by_ball['non_striker_y']+new_ball_by_ball['batsman_y']
new_ball_by_ball.drop(['batsman_y','non_striker_y'], axis=1)
new_ball_by_ball = new_ball_by_ball.rename({'batsman_x': 'batsman', 'non_striker_x': 'non_striker'}, axis=1)  # new method

#Get the partnership at previous ball for each row 
new_ball_by_ball['partnership']=new_ball_by_ball.shift(periods=1)['batsman']+new_ball_by_ball.shift(periods=1)['non_striker']
new_ball_by_ball['partnership_rev']=new_ball_by_ball.shift(periods=1)['non_striker']+new_ball_by_ball.shift(periods=1)['batsman']

#Get original partnership at each ball
new_ball_by_ball['original_partnership']=new_ball_by_ball['batsman']+new_ball_by_ball['non_striker']

#Create a dictionary for opening batsmen in each innings
bat_man_opening={};
for i in opening_batsmen.id.unique():
    for j in range(1,3):
        df=opening_batsmen.query("id==@i and inning==@j").sort_values(by=['inning'])
        for x in range(len(df.index)):
            bat_man_opening.update({(i,j,df.batsman.values[x]):1})
            bat_man_opening.update({(i,j,df.non_striker.values[x]):2})

#Create a dictionary for other batsmen in each inning
bat_man={}; y=3
for i in new_ball_by_ball.id.unique():
    for j in range(1,3):
        df=new_ball_by_ball.query("id==@i and inning==@j").sort_values(by=['inning','over','ball'])
        for x in range(len(df.index)-1):
            if((df.original_partnership.values[x+1]!=df.partnership.values[x+1]) & (df.original_partnership.values[x+1]!=df.partnership_rev.values[x+1])):
                if(df.partnership.values[x+1].find(df.batsman.values[x+1])>=0):
                    bat_man.update({(i,j,df.non_striker.values[x+1]):y})
                    y+=1
                else:
                    bat_man.update({(i,j,df.batsman.values[x+1]):y})
                    y+=1
        y=3

#Combine opening and other batsman to create a batting line up dictionary
batting_line_up = {**bat_man, **bat_man_opening}      

#restructure dataframe to add batting position
new_ball_by_ball['con'] = tuple(zip(new_ball_by_ball.id.values, new_ball_by_ball.inning.values, new_ball_by_ball.batsman.values))
new_ball_by_ball['in_at'] = new_ball_by_ball.con.map(batting_line_up)
new_ball_by_ball=new_ball_by_ball.drop('con',axis=1)

#final dataset batting line up
batting_line_up_df=pd.DataFrame.from_dict(batting_line_up,orient='index')
batting_line_up_df[['id','inning','batsman']]=pd.DataFrame(batting_line_up_df.index.tolist(), index=batting_line_up_df.index)
batting_line_up_df = batting_line_up_df.rename({0: 'in_at'}, axis=1)  # new method
batting_line_up_df = batting_line_up_df[['id', 'inning', 'batsman','in_at']]
batting_line_up_df=batting_line_up_df.sort_values(by=['id','inning','in_at'])